1. 引用我自己寫的 helper function: query_from_db可以直接從DB獲取資料>
2. Import 會用到的 library
3. 定義了python request 連線的 header

In [6]:
# Import the pre-implement db query function
from db_func import query_from_db
import re
import requests
from bs4 import BeautifulSoup
import datetime
import time

headers = {'user-agent': 'Mozilla/5.0 (Macintosh Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36',  'Connection': 'close'}

##### 將資料print出來的function，方便debug。

In [7]:
def print_util(urls, processor):
    for url in urls:
        try:
            print()
            temp = processor(url[0])
            for key in temp.keys():
                print('{}: {}'.format(key.capitalize(), temp[key]))
            print()
            print()
        except Exception as e:
            print()
            print(e)
            print('ERROR URL:')
            print(url)
            print()

### 選出10個在DB之中有最多資料的新聞來源：
```sql
SELECT news_source,COUNT(*) 
FROM news_db.news_rss_feeds
GROUP BY news_source
ORDER BY 2 DESC
LIMIT 11;
```

|  news_source    | COUNT(*) |
|------------------|----------|
| MSN              | 14204    |
| 自由時報         | 4997     |
| 大紀元           | 4394     |
| 經濟日報         | 3589     |
| ~新浪台灣新聞中心~ | ~3323~     |
| PCHOME           | 3213     |
| ETtoday          | 2585     |
| 新頭殼要聞       | 2117     |
| Rti 中央廣播電臺 | 2010     |
| Yahoo!奇摩股市   | 1976     |
| 公視新聞 |1796|  
最後因為新浪新聞有很多url是無效的，因此沒有放入 DB

### 1. 隨機選出50個Yahoo的新聞連結：
在開發時我是確保50個新聞連結都通過，但為了怕notebook篇幅太長，我改成五個來demo
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = 'Yahoo!奇摩股市' ORDER BY RAND()
LIMIT 50;
```

In [8]:
random_yahoo_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = 'Yahoo!奇摩股市' ORDER BY RAND()
LIMIT 5; """
yahoo_test_urls = query_from_db(random_yahoo_sql)

In [9]:

def yahoo_content_processor(url):
    res_dict = {}

    r = requests.get(url, headers = headers)
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")
    for br in soup.find_all("br"):
        br.replace_with("\n")
    title_tag = soup.find("title")
    if title_tag:
        title_category = title_tag.string.split(' - ')
        res_dict['news_title'] = title_category[0]
        if len(title_category) > 1:
            res_dict['news_category'] = title_category[1]
    fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    if fb_app_tag:
        res_dict['news_fb_app_id'] = str(fb_app_tag['content'])

    fb_page_tag = soup.find('meta', attrs = {'property':'fb:pages'})
    if fb_page_tag:
        res_dict['news_fb_page'] = str(fb_page_tag['content'])

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'news_keywords'})
    if keywords_tag:
        res_dict['news_keywords'] = keywords_tag['content']

    description_tag = soup.find('meta', attrs = {'name': 'description'})
    if description_tag:
        res_dict['news_description'] = description_tag['content']

    time_tag = soup.find('time')
    if time_tag:
        try:
            d1 = datetime.datetime.strptime(time_tag['datetime'], "%Y-%m-%dT%H:%M:%S.%fZ")
            db_date_format = '%Y-%m-%d %H:%M:%S'
            res = d1.strftime(db_date_format)
            res_dict['news_published_date'] = res
        except Exception as e:
            logging.error(e)
            print(e)
            pass

    article_body_tag = soup.find('article', attrs = {'itemprop':'articleBody'})
    temp_content = []
    links = []
    links_descs = []
    if article_body_tag:
        p_tags = article_body_tag.find_all('p')
        a_tags = article_body_tag.find_all('a')
        if p_tags:
            for index, p in enumerate(p_tags):
                if p.get('content'):
                    temp_content.append(p.get_text().strip())
        if a_tags:
            for a in a_tags:
                if a.get_text().strip():
                    links.append(a['href'])
                    links_descs.append(a.get_text().strip())
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs

    if temp_content:    
        # Capture the description start with 公告 to content
        if title_category[0][:4] == '【公告】':
            prefix = title_category[0]
        else:
            prefix = ''
        content = prefix + ' '.join(temp_content).replace('。 ', '。\n')
        res_dict['news'] = content
        return res_dict
    else:
        return 

In [10]:
print_util(yahoo_test_urls, yahoo_content_processor)


News_title: 今彩539第109136期開獎
News_category: Yahoo奇摩股市
News_fb_app_id: 399384933466174
News_fb_page: 114062348679049
News_description: （中央社台北2020年6月6日電）今彩539第109136期開獎，中獎號碼29、39、31、35、19。3星彩中獎號碼009，4星彩中獎號碼1576。實際中獎獎號以台彩公布為準。1090606
News_published_date: 2020-06-06 12:50:51
News: （中央社台北2020年6月6日電）今彩539第109136期開獎，中獎號碼29、39、31、35、19。3星彩中獎號碼009，4星彩中獎號碼1576。
實際中獎獎號以台彩公布為準。1090606



News_title: 《國際金融》搶進新興市場時機到？分析師揭2風險滅火
News_category: Yahoo奇摩股市
News_fb_app_id: 399384933466174
News_fb_page: 114062348679049
News_keywords: 新興市場
News_description: 【時報-台北電】隨著疫情和緩，油價逐步回升，加上市場資金充沛，讓許多人將眼光投向了新興市場，希望在此找到新天地，不過，專家認為，「全球復甦狀況低於預期」以及「美陸關係緊張」，恐將成為新興市場的兩大壓力。 巴隆周刊報導，BCA Research首席新興市場分析師Arthur Budaghyan表示，儘管近期市場資金蜂擁進駐新興市場，但全球復甦狀況低於市場預期與陸美關係緊張，仍將成為新興市場的兩大風險。 ...
News_published_date: 2020-06-05 09:14:24
News: 【時報-台北電】隨著疫情和緩，油價逐步回升，加上市場資金充沛，讓許多人將眼光投向了新興市場，希望在此找到新天地，不過，專家認為，「全球復甦狀況低於預期」以及「美陸關係緊張」，恐將成為新興市場的兩大壓力。
巴隆周刊報導，BCA Research首席新興市場分析師Arthur Budaghyan表示，儘管近期市場資金蜂擁進駐新興市場，但全球復甦狀況低於市場預期與陸美關係緊張，仍將成為新興市場的兩大風險。
Bud

### 2. 隨機選出50個 MSN 的新聞連結：
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = 'MSN' ORDER BY RAND()
LIMIT 50;
```

In [11]:
random_msn_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = 'MSN' ORDER BY RAND()
LIMIT 5; """
msn_test_urls = query_from_db(random_msn_sql)

In [12]:

def msn_content_processor(url):
    res_dict = {}
    prefix = ['相關報導', '※', '＊', '更多三立新聞網報導', '延伸閱讀', '資料來源', '更多中時電子報精彩報導', '看了這篇文章的人，也']
    pattern = re.compile(r'^{}'.format('|'.join(prefix)))
    
    if url.startswith('https://www.msn.com/zh-tw/video/'):
        return
    r = requests.get(url, headers = headers)
    r.encoding='utf-8'
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")

    if not soup:
        return 
    title_tag = soup.find("title")
    if title_tag:
        res_dict['news_title'] = title_tag.text.strip()
        
    
    category_tag = soup.find('div', attrs = {'class':'logowrapper'})
    if category_tag:
        _, category = category_tag.get_text().strip().split('\n')
        res_dict['news_category'] = category
        
    description_tag = soup.find('meta', attrs = {'name': 'description'})
    if description_tag:
        res_dict['news_description'] = description_tag['content']

    time_tag = soup.find('div', attrs = {'class': 'timeinfo-txt'})
    if time_tag:
        date_time_tag = time_tag.find('time')
        try:
            d1 = datetime.datetime.strptime(date_time_tag['datetime'], "%Y-%m-%dT%H:%M:%S.000Z") 
            d1 -= datetime.timedelta(hours=8)
            db_date_format = '%Y-%m-%d %H:%M:%S'
            date_res = d1.strftime(db_date_format)
            res_dict['news_published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(date_time_tag['datetime'], "%Y-%m-%dT%H:%M:%SZ") 
                d1 -= datetime.timedelta(hours=8)
                db_date_format = '%Y-%m-%d %H:%M:%S'
                date_res = d1.strftime(db_date_format)
                res_dict['news_published_date'] = date_res
            except Exception as e2:
                content_parser.logger.info('MSN date error, url: {}'.format(url))
    
    article_body_tag = soup.find('section', attrs = {'itemprop':'articleBody'})
    temp_content, links, links_descs = [], [], []
    if article_body_tag:
        p_tags = article_body_tag.find_all('p')
        div_tags = article_body_tag.find_all('div')
        a_tags = article_body_tag.find_all('a')
        if p_tags:
            for p in p_tags:
                #print(p.get_text())
                # Ignore the image caption
                if p.find('span'):
                    p.span.decompose()
                if p.find('a'):
                    p.a.decompose()
                if p.text.strip():
                    if pattern.match(p.text.strip()):
                        break
                    elif p.text.strip()[0] in ('▲','▼'):
                        continue
                    temp_content.append(p.text.strip())
        elif div_tags:
            for div in div_tags:
                if div.find('span'):
                    div.span.decompose()
                if div.find('a'):
                    div.a.decompose()
                if div.text.strip():
                    if pattern.match(div.text.strip()):
                        break
                    elif div.text.strip()[0] in ('▲','▼'):
                        continue
                    temp_content.append(div.text.strip())
            
        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(a.get_text().strip())
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs

    if len(temp_content):
        content = '\n'.join(temp_content)
        res_dict['news'] = content

    if not res_dict or 'news' not in res_dict: 
        #content_parser.logger.error('MSN url: {} did not process properly'.format(url))
        print('MSN url: {} did not process properly'.format(url))
        return 
    return res_dict

In [13]:
print_util(msn_test_urls, msn_content_processor)


News_title: 你正悄悄脫水中？ 6大原因讓你意想不到
News_category: 生活
News_description: 天氣超熱，你喝水了嗎？身體一旦缺水包括皮膚狀況、新陳代謝都受影響，還可能令人煩躁焦慮。只是除了曝曬、運動等因素，還有些脫水原因，可能讓你意想不到。 人體60%都是水分，根據康乃狄克大學（University of Connecticut）的研究，光是少掉1.5%的水分，情緒、活力、認知功能都會受衝擊。 脫水表示人體失去的水分多於攝取的水分，依程度分為輕度至重度，輕度脫水可自行居家護理，嚴重的話則可能要就醫。不過一般想到脫水，只覺得和水喝太少、流汗、太陽太大有關，其實脫水背後的因素包括年紀、飲食、甚至疾病： 1. 生理期 一般不會想到月經也可能導致脫水，但經期絕對要多多補充水分。雌激素與黃體素都會影響體內含水量，尤其有經前症候群的人，更需要多喝水。有些女性經血量特別多，連帶剝奪體內水分，如果你更換衛生棉的頻率超過每2小時1次，就要注意經期脫水的可能性。 2. 低醣飲食...
News_published_date: 2020-06-04 00:57:27
News_related_url: ['https://www.commonhealth.com.tw/article/article.action?nid=75597&utm_source=msn&utm_medium=referral&utm_campaign=msn', 'https://www.commonhealth.com.tw/article/article.action?nid=81409&utm_source=msn&utm_medium=referral&utm_campaign=msn', 'https://www.commonhealth.com.tw/article/article.action?nid=64963&utm_source=msn&utm_medium=referral&utm_campaign=msn', 'https://www.commonhealth.com.tw/article/article.action?nid=61963&utm_source=msn&utm_medium=referral&utm_campaign=msn',

### 3. 隨機選出50個自由時報的新聞連結：
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = '自由時報' ORDER BY RAND()
LIMIT 50;
```

In [15]:
ltn_msn_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = '自由時報' ORDER BY RAND()
LIMIT 5; """
ltn_test_urls = query_from_db(ltn_msn_sql)

In [16]:
def ltn_content_processor(url):
    postfix = ["""一手掌握經濟脈動
    點我訂閱自由財經Youtube頻道""", 
    """☆民眾如遇同居關係暴力情形，可撥打113保護專線，或向各地方政府家庭暴力防治中心求助☆""", 
    """☆健康新聞不漏接，按讚追蹤粉絲頁☆更多重要醫藥新聞訊息，請上自由健康網""",
    """《自由開講》是一個提供民眾對話的電子論壇，不論是對政治、經濟或社會、文化等新聞議題，有意見想表達、有話不吐不快，都歡迎你熱烈投稿。文長700字內為優，來稿請附真實姓名（必寫。有筆名請另註）、職業、聯絡電話、E-mail帳號。本報有錄取及刪修權，不付稿酬；請勿一稿多投，錄用與否將不另行通知。投稿信箱：LTNTALK@gmail.com""",
    """「武漢肺炎專區」請點此，更多相關訊息，帶您第一手掌握。""",
    """"""]
    res_dict = {}
    #headers = {'user-agent': 'Mozilla/5.0 (Macintosh Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36',  'Connection': 'close'}
    r = requests.get(url, headers = headers)
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")
    if not soup:
        return 
    title_tag = soup.find("title")
    if title_tag:
        title_category = title_tag.string.split(' - ')
        res_dict['news_title'] = title_category[0].strip()
        if len(title_category) > 1:
            res_dict['news_category'] = title_category[1].strip()
        else:
            try:
                res_dict['news_title'], res_dict['news_category'] = title_tag.string.split('|')
            except:
                pass
    fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    if fb_app_tag:
        res_dict['news_fb_app_id'] = str(fb_app_tag['content'])

    fb_page_tag = soup.find('meta', attrs = {'property':'fb:pages'})
    if fb_page_tag:
        res_dict['news_fb_page'] = str(fb_page_tag['content'])

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'news_keywords'})
    if keywords_tag:
        res_dict['news_keywords'] = keywords_tag['content']

    description_tag = soup.find('meta', attrs = {'name': 'description'})
    if description_tag:
        res_dict['news_description'] = description_tag['content']

    time_tag_1 = soup.find('meta', attrs = {'property': 'article:published_time'})
    time_tag_2 = soup.find('meta', attrs = {'name': 'pubdate'})
    if time_tag_1:
        try:
            d1 = datetime.datetime.strptime(time_tag_1.get('content'), "%Y-%m-%dT%H:%M:%S+08:00") 
            d1 -= datetime.timedelta(hours=8)
            db_date_format = '%Y-%m-%d %H:%M:%S'
            date_res = d1.strftime(db_date_format)
            res_dict['news_published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(time_tag_1.get('content'), "%Y-%m-%dT%H:%M:%SZ") 
                d1 -= datetime.timedelta(hours=8)
                db_date_format = '%Y-%m-%d %H:%M:%S'
                date_res = d1.strftime(db_date_format)
                res_dict['news_published_date'] = date_res
            except Exception as e2:
                content_parser.logger.info('LTN date error {}, URL: {}'.format(e2, url))

    elif time_tag_2:
        try:
            d1 = datetime.datetime.strptime(time_tag_2.get('content'), "%Y-%m-%dT%H:%M:%S+08:00")
            d1 -= datetime.timedelta(hours=8)
            db_date_format = '%Y-%m-%d %H:%M:%S'
            date_res = d1.strftime(db_date_format)
            res_dict['news_published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(time_tag_2.get('content'), "%Y-%m-%dT%H:%M:%SZ") 
                d1 -= datetime.timedelta(hours=8)
                db_date_format = '%Y-%m-%d %H:%M:%S'
                date_res = d1.strftime(db_date_format)
                res_dict['news_published_date'] = date_res
            except Exception as e2:
                content_parser.logger.info('LTN date error: {}, URL:{}'.format(e2, url))

    article_body_tag = soup.find('div', attrs = {'itemprop':'articleBody'})
    article_body_tag2 = soup.find('div', attrs = {'class':'text boxTitle boxText'})
    temp_content = []
    links = []
    links_descs = []

    if article_body_tag:
        p_tags = article_body_tag.find_all('p', attrs = {'class': None})
        a_tags = article_body_tag.find_all('a')
        if p_tags:
            for p in p_tags:
                if p.find('span'):
                    p.span.decompose()
                if p.text:
                    if p.text.strip() not in postfix :
                        temp_content.append(p.text.strip())
                    else:
                        break
        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(a.get_text().strip())
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs
    elif article_body_tag2:
        p_tags = article_body_tag2.find_all('p', attrs = {'class': None})
        a_tags = article_body_tag2.find_all('a')
        if p_tags:
            temp = []
            for p in p_tags:
                if p.find('span'):
                    p.span.decompose()
                if p.text:
                    if p.text.strip() not in postfix :
                        temp_content.append(p.text.strip())
                    else:
                        break
        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(a.get_text().strip())
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs

    content = '\n'.join(temp_content).strip()
    if content:
        res_dict['news'] = content

    if not res_dict or 'news' not in res_dict:
        content_parser.logger.error('LTN url: {} did not process properly'.format(url))
        return
    return res_dict

In [17]:
print_util(ltn_test_urls,ltn_content_processor)


News_title: 成功預言韓國瑜遭三殺！ 李正皓抖出「郭董落淚」內幕
News_category: 自由娛樂
News_fb_app_id: 140490219413038
News_fb_page: 524751764333507
News_keywords: 李正皓, 韓國瑜, 自由娛樂, ltn
News_description: 〔記者徐郁雯／台北報導〕2018年韓國瑜以超強韓流之姿，選上高雄市長，只是他落跑選總統，不到2年光景，韓流終於被摧毀殆盡。曾是藍營名嘴的李正皓，因批評韓國瑜遭開除黨籍，如今他也悲憤吐露內情：「當時我被開除時，一群韓粉指著我的鼻子對著我罵背骨、叛徒。」表示當時是不想看國民黨被「三殺」，沒想到仍挽回不了
News_published_date: 2020-06-07 06:49:46
News_related_url: []
News_related_url_desc: []
News: 〔記者徐郁雯／台北報導〕2018年韓國瑜以超強韓流之姿，選上高雄市長，只是他落跑選總統，不到2年光景，韓流終於被摧毀殆盡。曾是藍營名嘴的李正皓，因批評韓國瑜遭開除黨籍，如今他也悲憤吐露內情：「當時我被開除時，一群韓粉指著我的鼻子對著我罵背骨、叛徒。」表示當時是不想看國民黨被「三殺」，沒想到仍挽回不了悲劇。
李正皓表示，「去年，郭台銘董事長、我以及許多藍營朋友都曾公開預言，韓國瑜如果參選總統，唯一的下場就是讓國民黨被三殺（總統、立委、市長罷免），如今全應驗了。 」也說他們沒有超能力，只是有道德勇氣，認為腦袋正常的人，都知道當市長三個月落跑下場，只是國民黨不願正視。
李正皓指出，「去年的國民黨從下到上都瀰漫著一股投機的氣息」，認為韓國瑜一人可以救全黨，揭露去年郭台銘落淚，並非有求韓國瑜，「而是一種預知了國民黨被三殺但又無力回天的心急。」談及被開除黨籍，李正皓指出，韓粉狂罵他背骨，但他寧願賭上前途，也要指出韓國瑜不適任原因，是因為不想看國民黨被三殺。
如今，三殺預言成真，李正皓沉痛表示：「韓國瑜與國民黨的投機導致了今天的悲劇。」更說未來20年，國民黨在南台灣沒有可能再起，而當時捧韓的人全都裝死消失，「那所有的應報最後會誰來承擔，還是『國民黨』這三個字來承擔。 」預言國民黨這個百年招牌，將被一群投機份子玩到一文不值。



News_title: 波蘭

### 4. 隨機選出50個大紀元的新聞連結：
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = '大紀元' ORDER BY RAND()
LIMIT 50;
```

In [19]:
epoch_test_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = '大紀元' ORDER BY RAND()
LIMIT 5; """
epoch_test_urls = query_from_db(epoch_test_sql)

In [20]:
def epoch_content_processor(url):
    res_dict = {}
    r = requests.get(url, headers = headers)
    r.encoding='utf-8'
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")
    if not soup:
        return 
    title_tag = soup.find("title")
    if title_tag:
        title_category = title_tag.string.split(' | ', 1)
        res_dict['news_title'] = title_category[0]
        
    
    category_tag = soup.find('meta', attrs = {'property':'article:section'})
    if category_tag:
        res_dict['news_category'] = str(category_tag['content'])
        
        
    fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    if fb_app_tag:
        res_dict['news_fb_app_id'] = str(fb_app_tag['content'])

    fb_page_tag = soup.find('meta', attrs = {'property':'fb:pages'})
    if fb_page_tag:
        res_dict['news_fb_page'] = str(fb_page_tag['content'])

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'news_keywords'})
    if keywords_tag:
        res_dict['news_keywords'] = keywords_tag['content']

    description_tag = soup.find('meta', attrs = {'name': 'description'})
    if description_tag:
        res_dict['news_description'] = description_tag['content']

    time_tag = soup.find('meta', attrs = {'property': 'article:published_time'})

    if time_tag:
        try:
            d1 = datetime.datetime.strptime(time_tag.get('content'), "%Y-%m-%dT%H:%M:%S+08:00") 
            d1 -= datetime.timedelta(hours=8)
            db_date_format = '%Y-%m-%d %H:%M:%S'
            date_res = d1.strftime(db_date_format)
            res_dict['news_published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(time_tag.get('content'), "%Y-%m-%dT%H:%M:%SZ") 
                d1 -= datetime.timedelta(hours=8)
                db_date_format = '%Y-%m-%d %H:%M:%S'
                date_res = d1.strftime(db_date_format)
                res_dict['news_published_date'] = date_res
            except Exception as e2:
                print(e2)
                content_parser.logger.info('Epoch date error {}'.format(e2))

    article_body_tag = soup.find('div', attrs = {'itemprop':'articleBody'})
    #print(article_body_tag)
    temp_content = []
    links = []
    links_descs = []
    if article_body_tag:
        p_tags = article_body_tag.find_all('p', attrs = {'class': None})
        a_tags = article_body_tag.find_all('a')
        if p_tags:
            
            for p in p_tags:
                if p.find('script', attrs = {'type': 'text/javascript'}):
                    continue
                temp_content.append(p.get_text().strip())
                
        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(a.get_text().strip())
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs
    content = '\n'.join(temp_content).strip()
    if content:
        res_dict['news'] = content

    if not res_dict or 'news' not in res_dict:
        return
        content_parser.logger.error('Epoch url: {} did not process properly'.format(url))

    return res_dict


In [21]:
print_util(epoch_test_urls, epoch_content_processor)


News_title: 「政審」又來了！中國人答不答應？
News_category: 各地分網
News_fb_app_id: 1571885662850711
News_fb_page: 156786811025453
News_keywords: 政審,文革,熱點互動,大紀元
News_description: 中共的「政審」為何又回來了？中共建政以來是如何用「政審」來箝制民眾的？高校為何越來越成為中共進一步嚴控思想和言論的重地？ 今晚美東時間9:30，新唐人電視台熱點互動直播節目將邀請專家深度解析，歡迎觀眾朋友們屆時收看並參與討論。
News_published_date: 2018-11-14 19:37:49
News_related_url: ['https://www.epochtimes.com/b5/tag/%e6%94%bf%e5%af%a9.html', 'https://www.epochtimes.com/b5/tag/%e6%96%87%e9%9d%a9.html', 'https://www.epochtimes.com/b5/tag/%e6%94%bf%e5%af%a9.html', 'https://www.epochtimes.com/b5/tag/%e7%86%b1%e9%bb%9e%e4%ba%92%e5%8b%95.html']
News_related_url_desc: ['政審', '文革', '政審', '熱點互動']
News: 【大紀元2018年11月15日訊】日前，重慶、福建接連發布消息，將對2019年高考生進行「政審」，有反對「四項基本原則」、「反對憲法言行」者不能參加考試。消息引發輿論強烈反響，被批向文革倒退。網絡更是瘋傳一篇檄文，斥責中共官員不知道自己是誰。另一方面，廣西高校日前又傳出清查全校4萬多師生手機電腦內容的事件。
中共的「政審」為何又回來了？中共建政以來是如何用「政審」來箝制民眾的？高校為何越來越成為中共進一步嚴控思想和言論的重地？ 今晚美東時間9:30，新唐人電視台熱點互動直播節目將邀請專家深度解析，歡迎觀眾朋友們屆時收看並參與討論。
美東時間：週三（11月14日）晚9點半到10點10分
北京時間：週四（11月15日）早9點半到10點10分
歡迎在節目直播期間發手機簡訊即時發送您的評論或提

### 5. 隨機選出50個經濟日報的新聞連結：
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = '經濟日報' ORDER BY RAND()
LIMIT 50;
```

In [22]:
udn_test_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = '經濟日報' ORDER BY RAND()
LIMIT 5; """
udn_test_urls = query_from_db(udn_test_sql)

In [23]:
def udn_content_processor(url):
    res_dict = {}
    r = requests.get(url, headers = headers)
    r.encoding='utf-8'
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")
    if not soup:
        return 
    title_tag = soup.find("title")
    if title_tag:
        title_category_lst = title_tag.string.split(' | ', 3)
        res_dict['news_title'] = title_category_lst[0]
        try:
            res_dict['news_category'] = title_category_lst[2]
        except:
            pass
        
        
        
    fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    if fb_app_tag:
        res_dict['news_fb_app_id'] = str(fb_app_tag['content'])

    fb_page_tag = soup.find('meta', attrs = {'property':'fb:pages'})
    if fb_page_tag:
        res_dict['news_fb_page'] = str(fb_page_tag['content'])

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'news_keywords'})
    if keywords_tag:
        res_dict['news_keywords'] = keywords_tag['content']

    description_tag = soup.find('meta', attrs = {'name': 'description'})
    if description_tag:
        res_dict['news_description'] = description_tag['content']

    time_tag = soup.find('div', attrs = {'class': 'shareBar__info--author'})

    if time_tag:
        try:

            d1 = datetime.datetime.strptime(time_tag.find('span').text, "%Y-%m-%d %H:%M")
            print(d1)
            d1 -= datetime.timedelta(hours=8)
            db_date_format = '%Y-%m-%d %H:%M:%S'
            date_res = d1.strftime(db_date_format)
            res_dict['news_published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(time_tag.get('content'), "%Y-%m-%d %H:%M:%S") 
                d1 -= datetime.timedelta(hours=8)
                db_date_format = '%Y-%m-%d %H:%M:%S'
                date_res = d1.strftime(db_date_format)
                res_dict['news_published_date'] = date_res
            except Exception as e2:
                print(e2)
                content_parser.logger.info('Epoch date error {}'.format(e2))
    article_body_tag = soup.find('div', attrs = {'id':'article_body'})
    content_temp, links, links_descs = [], [], []
    if article_body_tag:
        p_tags = article_body_tag.find_all('p', attrs = {'class': None})
        a_tags = article_body_tag.find_all('a')
        if p_tags:
            for p in p_tags:
                if p.get_text().strip():
                    content_temp.append(p.get_text().strip())
        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(a.get_text().strip())
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs
                
    content = '\n'.join(content_temp).strip()
    if content:
        res_dict['news'] = content

    if not res_dict or 'news' not in res_dict:
        return
        content_parser.logger.error('Epoch url: {} did not process properly'.format(url))

    return res_dict


In [24]:
print_util(udn_test_urls, udn_content_processor)


2019-12-11 15:18:00
News_title: 隱藏版台灣之光  冷鏈物流技術輸出國際
News_category: 商情
News_fb_app_id: 267846550050865
News_fb_page: 179043295447791
News_description: 台灣便利商店與超市林立，鮮食唾手可得，背後功臣是完善的冷鏈系統，台灣冷鏈產業上、下游串接完整，經驗豐富，相中新南向新崛起的冷鏈廣大商機，由外貿協會邀請來自越南、印尼、馬來西亞及緬甸…等國買主來台，實地
News_published_date: 2019-12-11 07:18:00
News_related_url: []
News_related_url_desc: []
News: 台灣便利商店與超市林立，鮮食唾手可得，背後功臣是完善的冷鏈系統，台灣冷鏈產業上、下游串接完整，經驗豐富，相中新南向新崛起的冷鏈廣大商機，由外貿協會邀請來自越南、印尼、馬來西亞及緬甸…等國買主來台，實地參訪交流，協助業者布局東南亞冷鏈市場，進一步將台灣優勢的冷鏈技術服務輸出全球。
貿協會秘書長葉明水（中）與越南、印尼、馬來西亞及緬甸等5國6家經營冷鏈物流服務、生鮮產品經銷及農水產養殖等海外買主合照。  貿協／提供facebook
為提升台灣冷鏈物流國際輸出能量，在經濟部國際貿易局的支持下，外貿協會與台灣冷鏈協會及中華民國物流協會攜手合作，10日特別舉辦「台灣冷鏈新南向商機研討交流會」，邀請6家經營冷鏈物流服務、生鮮產品經銷及農水產養殖等9位海外買主，吸引138家台灣業者報名參與。
會後，大會安排外商實地參訪國內冷鏈產業鍊代表性業者研華、永聯物流、廣運機械及全日物流等企業，完整呈現台灣冷凍保鮮、控冷技術及相關跨境整合的客製化輸出服務。
外貿協會秘書長葉明水致詞表示，台灣冷鏈產業在國際上具競爭力，在協助出口台灣優質生鮮蔬果上扮演著重要的角色，最近台灣與美國已簽署「台灣產番石榴鮮果實輸美工作計畫」，台灣是繼墨西哥後、全球第二個芭樂可以銷美的國家，更是亞洲第一，首批預計12月底前運美。
會中，日本NEC Solution Innovators, Ltd.及印尼冷鏈協會主席（Indonesia Cold Chain Association；ICCA）發表專題演講。NEC積極研究Smart Cold 

### 6. 隨機選出50個新浪台灣新聞中心的新聞連結： (skip this phase, too many error urls)
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = '新浪台灣新聞中心' ORDER BY RAND()
LIMIT 50;
```

In [ ]:
sina_test_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = '新浪台灣新聞中心' ORDER BY RAND()
LIMIT 50; """
sina_test_urls = query_from_db(sina_test_sql)

In [ ]:

def sina_content_processor(url):
    res_dict = {}
    r = requests.get(url, headers = headers)
    r.encoding='utf-8'
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")
    if not soup:
        return 
    title_tag = soup.find("title")
    if title_tag:
        title_category = title_tag.string.rsplit('-', 2)
        res_dict['news_title'] = title_category[0].strip()
        if len(title_category) > 1:
            res_dict['news_category'] = title_category[-1].strip()
    
    
    # Sina did not have FB available
    #fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    #if fb_app_tag:
    #    res_dict['news_fb_app_id'] = str(fb_app_tag['content'])
    
    #fb_page_tag = soup.find('meta', attrs = {'property':'fb:pages'})
    #if fb_page_tag:
    #    res_dict['news_fb_page'] = str(fb_page_tag['content'])

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'keywords'})
    if keywords_tag:
        res_dict['news_keywords'] = keywords_tag['content']

    description_tag = soup.find('meta', attrs = {'name': 'description'})
    if description_tag:
        res_dict['news_description'] = description_tag['content']
        
    time_tag = soup.find('meta', attrs = {'name': 'article:published_time'})

    if time_tag:
        try:
            d1 = datetime.datetime.strptime(time_tag.get('content'), "%Y-%m-%dT%H:%M:%S+08:00") 
            d1 -= datetime.timedelta(hours=8)
            db_date_format = '%Y-%m-%d %H:%M:%S'
            date_res = d1.strftime(db_date_format)
            res_dict['news_published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(time_tag.get('content'), "%Y-%m-%dT%H:%M:%SZ") 
                d1 -= datetime.timedelta(hours=8)
                db_date_format = '%Y-%m-%d %H:%M:%S'
                date_res = d1.strftime(db_date_format)
                res_dict['news_published_date'] = date_res
            except Exception as e2:
                print(e2)
                content_parser.logger.info('Sina date error {}, URL: {}'.format(e2, url))

    article_body_tag = soup.find('div', attrs = {'id':'article_content'})
    if article_body_tag:
        p1_tags = article_body_tag.find_all('p')
        if p1_tags:
            temp = []
            for p in p1_tags:
                if p.text:
                    temp.append(p.text.strip())
            content = '\n'.join(temp).strip()
            if content:
                res_dict['news'] = content


    if not res_dict or 'news' not in res_dict:
        return
        content_parser.logger.error('Sina url: {} did not process properly'.format(url))

    return res_dict


In [ ]:
print_util(sina_test_urls, sina_content_processor)

### 7. 隨機選出50個PCHOME的新聞連結：
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = 'PCHOME' ORDER BY RAND()
LIMIT 50;
```

In [25]:
pchome_test_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = 'PCHOME' ORDER BY RAND()
LIMIT 5; """
pchome_test_urls = query_from_db(pchome_test_sql)

In [26]:

def pchome_content_processor(url):
    res_dict = {}
    r = requests.get(url, headers = headers)
    r.encoding='utf-8'
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")
    if not soup:
        return
    for br in soup.find_all("br"):
        br.replace_with("\n")
    title_tag = soup.find("title")
    if title_tag:
        title_category = title_tag.string.rsplit('-', 2)
        res_dict['news_title'] = title_category[0].strip()
        if len(title_category) > 1:
            res_dict['news_category'] = title_category[-2].strip()
            
    fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    if fb_app_tag:
        res_dict['news_fb_app_id'] = str(fb_app_tag['content'])
    
    fb_page_tag = soup.find('meta', attrs = {'property':'fb:pages'})
    if fb_page_tag:
        res_dict['news_fb_page'] = str(fb_page_tag['content'])

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'keywords'})
    if keywords_tag:
        res_dict['news_keywords'] = keywords_tag['content']

    description_tag = soup.find('meta', attrs = {'name': 'description'})
    if description_tag:
        res_dict['news_description'] = description_tag['content']

    time_tag = soup.find('time')

    if time_tag:
        try:
            d1 = datetime.datetime.strptime(time_tag.get('pubdate'), "%Y-%m-%d %H:%M:%S") 
            d1 -= datetime.timedelta(hours=8)
            db_date_format = '%Y-%m-%d %H:%M:%S'
            date_res = d1.strftime(db_date_format)
            res_dict['news_published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(time_tag.get('pubdate'), "%Y-%m-%d %H:%M:%SZ") 
                d1 -= datetime.timedelta(hours=8)
                db_date_format = '%Y-%m-%d %H:%M:%S'
                date_res = d1.strftime(db_date_format)
                res_dict['news_published_date'] = date_res
            except Exception as e2:
                print(e2)
                content_parser.logger.info('PChome date error {}, URL: {}'.format(e2, url))

    article_body_tag = soup.find('div', attrs = {'calss':'article_text'})
    if article_body_tag:
        content = article_body_tag.text.strip()
        a_tags = article_body_tag.find_all('a')
        if content:
            content = re.sub('(\n)+', '\n', content)
            content = re.sub(r'(相關新聞[\s\S]+)', '', content)
            res_dict['news'] = content 
            
    if not res_dict or 'news' not in res_dict:
        return
        content_parser.logger.error('PChome url: {} did not process properly'.format(url))

    return res_dict

In [27]:
print_util(pchome_test_urls, pchome_content_processor)


News_title: 送別吳朋奉！茄子蛋淚灑會場 吳念真嘆：欠你一個男主角
News_category: 娛樂新聞
News_fb_app_id: 498546706899526
News_fb_page: 1423378217880597
News_keywords: 吳念真,吳慷仁,吳朋奉,鄭麗君
News_description:  影帝吳朋奉上個月25日，突然腦中風猝逝，享年55歲。今（7）日在板橋殯儀館舉行告別式，藝人王彩樺、鍾欣凌、吳慷仁、陳竹昇、游安順、林美秀、楊貴媚、莫子儀、喜翔；導演吳念真、魏德聖、王育麟、...
News_published_date: 2020-06-07 08:03:56
News: 吳朋奉今（7）日舉辦告別式。（圖 / 金熙國際提供）
影帝吳朋奉上個月25日，突然腦中風猝逝，享年55歲。今（7）日在板橋殯儀館舉行告別式，藝人王彩樺、鍾欣凌、吳慷仁、陳竹昇、游安順、林美秀、楊貴媚、莫子儀、喜翔；導演吳念真、魏德聖、王育麟、前文化部長鄭麗君都現身弔唁，會場播放著吳朋奉曾經的作品畫面，由荒山亮獻上一曲，親友們悲痛不捨。
王彩樺低調不受訪。（圖 / 記者林調遜攝）
陳竹昇現身弔唁。（圖 / 記者林調遜攝）
茄子蛋離開會場悲痛落淚。（圖 / 記者林調遜攝）告別式現場由吳念真致詞，他說最喜歡吳朋奉的眼睛，「桀驁不馴、又深情款款，演技真的不用說。」憶起當初看到茄子蛋的《浪子回頭》MV，吳念真還私訊吳朋奉誇讚他最後12秒情緒滿滿的演技，吳朋奉回他：「謝謝，演員不就靠這吃飯的嗎」，表達出他的自信、自在，而茄子蛋三位團員今日也現身會場，淚流滿面，表達出對朋奉哥的不捨。
吳念真在吳朋奉告別式上致詞。（圖 / 記者林調遜攝）吳念真透露，一直欠吳朋奉一個主角角色，沒想一拖再拖他就不在了，令人不勝唏噓。紙風車劇團執行長李永豐也為吳朋奉好友，吳念真在致詞時提到，問李永豐想對吳朋奉說什麼，李永豐則幽默提到紙風車排練場日前遭遇火災一事，「整個劇場都燒給你了，要好好演哦」。
吳慷仁（右二）曾和吳朋奉合作《出境事務所》。（圖 / 記者林調遜攝）而吳朋奉生前的愛犬「碰碰」也在會場送別主人最後一程，親友抱著牠致敬三鞠躬，吳朋奉家屬遵循吳朋奉生前遺願，採取環保樹葬方式，安葬在新北市三芝櫻花園區，並於6月18日舉行安葬儀式，但據悉當天只開放家族親友參加，18日後才會

### 8. 隨機選出50個Ettoday的新聞連結：
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = 'ETtoday' ORDER BY RAND()
LIMIT 50;
```

In [28]:
ettoday_test_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = 'ETtoday' ORDER BY RAND()
LIMIT 5; """
ettoday_test_urls = query_from_db(ettoday_test_sql)

In [29]:
def ettoday_content_processor(url):
    res_dict = {}
    r = requests.get(url, headers = headers)
    r.encoding='utf-8'
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")
    if not soup:
        return 
    title_tag = soup.find("title")
    if title_tag:
        title_category = title_tag.string.split(' | ')
        res_dict['news_title'] = title_category[0]
        if len(title_category) > 1:
            res_dict['news_category'] = title_category[1].strip()
    fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    if fb_app_tag:
        res_dict['news_fb_app_id'] = str(fb_app_tag['content'])
    # Ettoday did not put fb page tag on the content
    #fb_page_tag = soup.find('meta', attrs = {'property':'fb:admins'})
    #if fb_page_tag:
        #res_dict['news_fb_page'] = str(fb_page_tag['content'])

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'news_keywords'})
    if keywords_tag:
        res_dict['news_keywords'] = keywords_tag['content']

    description_tag = soup.find('meta', attrs = {'name': 'description'})
    if description_tag:
        res_dict['news_description'] = description_tag['content']

    time_tag = soup.find('meta', attrs = {'property': 'article:published_time'})

    if time_tag:
        try:
            d1 = datetime.datetime.strptime(time_tag.get('content'), "%Y-%m-%dT%H:%M:%S+08:00") 
            d1 -= datetime.timedelta(hours=8)
            db_date_format = '%Y-%m-%d %H:%M:%S'
            date_res = d1.strftime(db_date_format)
            res_dict['news_published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(time_tag.get('content'), "%Y-%m-%dT%H:%M:%SZ") 
                d1 -= datetime.timedelta(hours=8)
                db_date_format = '%Y-%m-%d %H:%M:%S'
                date_res = d1.strftime(db_date_format)
                res_dict['news_published_date'] = date_res
            except Exception as e2:
                print(e2)
                content_parser.logger.info('Ettoday date error {}, URL: {}'.format(e2, url))

    article_body_tag = soup.find('div', attrs = {'itemprop':'articleBody'})
    temp_content = []
    links, links_descs = [], []
    if article_body_tag:
        p_tags = article_body_tag.find_all('p', attrs = {'class': None})
        a_tags = article_body_tag.find_all('a')
        
        if p_tags:
            for p in p_tags:
                # Ignore the image caption
                if p.find('strong'):
                    continue
                if p.text:
                    temp_content.append(p.text.strip())
        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(a.get_text().strip().replace('\u3000', ' '))
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs
            
    content = '\n'.join(temp_content).strip()
    if content:
        res_dict['news'] = content

    if not res_dict or 'news' not in res_dict:
        return
        content_parser.logger.error('Ettoday url: {} did not process properly'.format(url))

    return res_dict

In [30]:
print_util(ettoday_test_urls, ettoday_content_processor)


News_title: 陳乃榮從背後靠上來！賴琳恩：偶爾閃一下　細肩帶「溢出深溝圓球」邪惡視角被看光
News_category: ETtoday星光雲
News_fb_app_id: 146858218737386
News_keywords: 賴琳恩,陳乃榮
News_description: 賴琳恩2017年初與交往多年的陳乃榮結婚，至今仍維持熱戀期的打情罵俏，已經許久沒曬恩愛的她，7日在社群網站PO文笑說：「你們說我都不放閃，好吧偶爾閃一下！」小倆口在餐桌前的甜蜜互動羨煞不少粉絲。(賴琳恩,陳乃榮)
News_published_date: 2020-06-07 06:14:00
News_related_url: ['https://www.etmall.com.tw/…/Final/…/SP_1115548/w/index.html…']
News_related_url_desc: ['►來讓「過期票券」變黃金！']
News: 記者潘慧中／綜合報導
賴琳恩2017年初與交往多年的陳乃榮結婚，至今仍維持熱戀期的打情罵俏，已經許久沒曬恩愛的她，7日在社群網站PO文笑說：「你們說我都不放閃，好吧偶爾閃一下！」小倆口在餐桌前的甜蜜互動羨煞不少粉絲。
照片中，可以看到賴琳恩放下及胸長捲髮，身穿細肩帶純白背心，從上往下拍的邪惡視角，剛好捕捉到豐滿北半球和深邃事業線，露出甜美笑容，舉手投足散發出濃濃女人味，陳乃榮則穿上同色系襯衫溫柔地靠在她身後。
久違的恩愛照連賴琳恩都忍不住笑說「閃到自己」，許多粉絲同樣被閃得很開心，「一閃就瞎眼等級」、「甜滋滋」、「眼睛要瞎了。」特別的是，居然有網友把陳乃榮誤認成《夫婦的世界》的李泰伍（朴海俊飾），她看了則笑回：「我也覺得超像，看完會不小心帶入戲中情緒這樣。」
事實上，賴琳恩曾透露陳乃榮個性貼心細膩，老公平時會創作寫歌，只要吵架就靠寫歌來挽回她的心，還會一起穿情侶裝出門約會，互動就像老夫老妻般恩愛。私下喜歡素顏出門的她，被問到老公是否喜歡沒有化妝的自己時，她害羞透露對方說過：「不管怎樣，你都很正。」滿足女生喜歡被稱讚的心情，最讓她感動的則是老公不時會準備小驚喜，「他經過花店看到漂亮的花，就會買給我，看到可愛的小玩偶長得像我也會買回來。」讓兩人始終維持戀愛熱度。



News_title: 謝和弦勒戒突收「大量生活用品」...

### 9. 隨機選出50個Rti 中央廣播電臺的新聞連結：
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = 'Rti 中央廣播電臺' ORDER BY RAND()
LIMIT 50;
```

In [31]:
rti_test_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = 'Rti 中央廣播電臺' ORDER BY RAND()
LIMIT 5; """
rti_test_urls = query_from_db(rti_test_sql)

In [32]:
def rti_content_processor(url):
    res_dict = {}
    r = requests.get(url, headers = headers)
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")
    reserved_keywords = ['Rti', '中央廣播電臺', 'Radio Taiwan International']
    title_tag = soup.find("title")
    if title_tag:
        title_category = title_tag.string.split(' - ', 1)
        res_dict['news_title'] = title_category[0]
    category_tag = soup.find('div', attrs = {'class':'swiper-wrapper'})
    if category_tag:
        
        res_dict['news_category'] = category_tag.find('a', attrs = {'class':'active'}).get_text().strip()

    fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    if fb_app_tag:
        res_dict['news_fb_app_id'] = str(fb_app_tag['content'])

    #fb_page_tag = soup.find('meta', attrs = {'property':'fb:pages'})
    #if fb_page_tag:
    #    res_dict['news_fb_page'] = str(fb_page_tag['content'])

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'keywords'})
    if keywords_tag:
        keyword_lst = [keyword for keyword in keywords_tag['content'].split(',') if keyword not in reserved_keywords]
        res_dict['news_keywords'] = ','.join(keyword_lst)

    description_tags = soup.find_all('meta', attrs = {'name': 'description'})
    if description_tags:
        res_dict['news_description'] = description_tags[1]['content']

    time_tag = soup.find('li', attrs = {'class': 'date'})
    if time_tag:
        try:
            d1 = datetime.datetime.strptime(time_tag.text, "時間：%Y-%m-%d %H:%M")
            d1 -= datetime.timedelta(hours=8)
            db_date_format = '%Y-%m-%d %H:%M:%S'
            date_res = d1.strftime(db_date_format)
            res_dict['news_published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(time_tag.text, "時間：%Y-%m-%d %H:%M:%S")
                d1 -= datetime.timedelta(hours=8)
                db_date_format = '%Y-%m-%d %H:%M:%S'
                date_res = d1.strftime(db_date_format)
                res_dict['news_published_date'] = date_res
            except Exception as e2:
                content_parser.logger.info('RTI url: {} date not process properly, error message {}'.format(url, e2))

    article_body_tag = soup.find('article', attrs = {'class' : None})
    temp_content, links, links_descs = [], [], []
    if article_body_tag:
        p_tags = article_body_tag.find_all('p', attrs = {'class': None})
        a_tags = article_body_tag.find_all('a')
        if p_tags:
            for p in p_tags:
                if p.text:
                    temp_content.append(p.text.strip())

        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(a.get_text().strip())
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs
            
    content = '\n'.join(temp_content).strip()
    if content:
        res_dict['news'] = content
    if not res_dict or 'news' not in res_dict:
        return
        content_parser.logger.error('RTI url: {} did not process properly'.format(url))
    return res_dict

In [33]:
print_util(rti_test_urls, rti_content_processor)


News_title: 美參議員：中國意圖減緩或破壞疫苗的研發
News_category: 國際
News_fb_app_id: 1722151021422152
News_keywords: 疫苗,武漢肺炎,COVID-19
News_description: 美國共和黨國會參議員史考特(Rick Scott)今天(7日)表示，美國有證據顯示，中國正企圖減緩或破壞西方國家對俗稱武漢肺炎的2019年冠狀病毒疾病(COVID-19)的疫苗研發。史考特向英國廣播公司電視台(BBC TV)表示，「我們必須完成疫苗研發工作。不幸的是，我們有證據顯示，共產中國正試圖破...
News_published_date: 2020-06-07 09:32:00
News: 美國共和黨國會參議員史考特(Rick Scott)今天(7日)表示，美國有證據顯示，中國正企圖減緩或破壞西方國家對俗稱武漢肺炎的2019年冠狀病毒疾病(COVID-19)的疫苗研發。
史考特向英國廣播公司電視台(BBC TV)表示，「我們必須完成疫苗研發工作。不幸的是，我們有證據顯示，共產中國正試圖破壞我們或減緩疫苗的研發。」
史考特說：「中國不要我們率先研發出疫苗，他們決心要成為美國人的勁敵，而我認為，中國想要成為全球民主國家的勁敵。」
在被問到美國已經掌握何種證據時，史考特拒絕透露詳情，只說這項證據來自情報界。
史考特表示，「疫苗對所有想要重新振興經濟的我們這些人來說，非常重要。我所確切相信的是，英國或是我們是否率先研發出疫苗，我們會和共產中國分享，但他們不會(和我們)分享。」



News_title: 批日本向中國屈服　球星本田圭佑為港發聲
News_category: 國際
News_fb_app_id: 1722151021422152
News_keywords: 港版國安法,本田圭佑
News_description: 香港立場新聞報導，日本足球明星本田圭佑今天(7日)在推特發文為香港發聲。報導表示，根據共同社，日本政府為改善與中國的關係，及避免妨礙中國國家主席習近平訪日，因此拒絕參與英國和美國提出的聯合聲明，批評港版國安法。本田圭佑在推特轉發了相關報導，他在貼文上批評日本政府，「為什麼日本拒絕參與批評中國的聲明！...
News_published_date: 2020-06-07 11:

### 10. 隨機選出50個公視新聞網的新聞連結：
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = '公視新聞網' ORDER BY RAND()
LIMIT 50;
```

In [34]:
pts_test_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = '公視新聞網' ORDER BY RAND()
LIMIT 5; """
pts_test_urls = query_from_db(pts_test_sql)

In [35]:
def pts_content_processor(url):
    res_dict = {}
    r = requests.get(url, headers = headers)
    r.encoding='utf-8'
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")

    title_tag = soup.find("title")
    if title_tag:
        title_category = title_tag.string.rsplit(' | ', 1)
        res_dict['news_title'] = title_category[0].strip()
            
    fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    if fb_app_tag:
        res_dict['news_fb_app_id'] = str(fb_app_tag['content'])
    

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'keywords'})
    if keywords_tag:
        res_dict['news_keywords'] = keywords_tag['content']

    description_tags = soup.find_all('meta', attrs = {'name': 'description'})
    if len(description_tags) > 1:
        res_dict['news_description'] = description_tags[1]['content']
        
    time_tag = soup.find('div', attrs = {'class': 'maintype-wapper hidden-lg hidden-md'})
    if time_tag:
        date_tag = time_tag.find('h2')
        if date_tag:
            try:
                d1 = datetime.datetime.strptime(date_tag.text, "%Y年%m月%d日") 
                #d1 -= datetime.timedelta(hours=8)
                db_date_format = '%Y-%m-%d'
                date_res = d1.strftime(db_date_format)
                res_dict['news_published_date'] = date_res
            except Exception as e1:
                print(e1)
                content_parser.logger.info('PTS date error {}, URL: {}'.format(e1, url))

    article_body_tag = soup.find('div', attrs = {'class':'article_content'})
    if article_body_tag:
        content = article_body_tag.text.strip()
        if content:
            res_dict['news'] = content
            
    if not res_dict or 'news' not in res_dict:
        return
        content_parser.logger.error('PTS url: {} did not process properly'.format(url))

    return res_dict

In [36]:
print_util(pts_test_urls, pts_content_processor)


News_title: 國際媒體同步關注罷韓案 各自提出解讀
News_fb_app_id: 466487386878484
News_keywords: 國際媒體同步關注罷韓案 各自提出解讀
News_description: 國際媒體都很關心罷免案的結果，華爾街日報指出，這是台灣選民將靠中國的市長趕落台；新加坡海峽時報則是形容，韓國瑜是被憤怒的選民罷免。...
News_published_date: 2020-06-07
News: 國際媒體都很關心罷免案的結果，華爾街日報指出，這是台灣選民將靠中國的市長趕落台；新加坡海峽時報則是形容，韓國瑜是被憤怒的選民罷免。

罷韓投票結果出爐，中國央視派出記者，在高雄市四維行政中心連線報導最新情形。央視隨後也播出了韓國瑜遭罷免的新聞，並分析在這一次的選舉後，國、民兩黨將繼續角力。央視記者說：「挺韓陣營有可能會提出更多的報復性罷免案，由此造成台灣內部的進一步分裂。」
國際媒體關注罷韓新聞，華爾街日報以「台灣民眾用選票將親中市長趕下台」為標題，形容韓國瑜是「台灣最知名的親中政治人物之一」。報導指出韓國瑜在五個月內，連續第二次失敗，象徵台灣人民第二度拒絕北京。
新加坡海峽時報也報導，台灣的「前總統候選人」韓國瑜 被憤怒的選民免職，報導也回顧了今年一月總統大選時，韓國瑜慘敗給民進黨對手蔡英文。兩岸交流協會秘書長王智盛說明，「去年下半年的總統大選，一直到今年的肺炎疫情。台灣社會內部反中情緒的投射，是有一定，有在這裡面的。」
對韓國瑜被罷免，香港眾志秘書長黃之鋒也在臉書發文，認為台灣人的抉擇，是拒絕一個走進中聯辦，對中共頻頻示好的市長。學者則分析，在總統大選後，反親中情緒仍餘波盪漾，韓國瑜遭罷免意味選民對親中路線的反感。勢必將讓兩岸未來，政治關係更加急凍，經濟與民間交流，恐怕也會緩步降溫。



News_title: 罷韓通過 蔡臉書：給所有政治人物最大警惕
News_fb_app_id: 466487386878484
News_keywords: 罷韓通過 蔡臉書：給所有政治人物最大警惕
News_description: 針對韓國瑜罷免成功，總統蔡英文昨在臉書表示，這個結果是給所有政治人物最大的警戒。新北市長侯友宜在今天呼籲，外界多給韓國瑜空間；台北市長柯文哲則表示，希望台灣社會可以冷靜。同時，國民黨再次出現改革...
N

### 11. 隨機選出50個新頭殼要聞的新聞連結：
```sql
SELECT news_url FROM news_rss_feeds
WHERE news_source = '新頭殼要聞' ORDER BY RAND()
LIMIT 50;
```

In [37]:
newstalk_test_sql = """
SELECT news_url FROM news_rss_feeds
WHERE news_source = '新頭殼要聞' ORDER BY RAND()
LIMIT 5; """
newstalk_test_urls = query_from_db(newstalk_test_sql)

In [38]:
def newstalk_content_processor(url):
    res_dict = {}
    r = requests.get(url, headers = headers)
    r.encoding='utf-8'
    web_content = r.text
    soup = BeautifulSoup(web_content, "lxml")
    if not soup:
        return 
    title_tag = soup.find("title")
    if title_tag:
        title_category = title_tag.string.split(' | ')
        res_dict['news_title'] = title_category[0]
        if len(title_category) > 1:
            res_dict['news_category'] = title_category[1]
    fb_app_tag = soup.find('meta', attrs = {'property':'fb:app_id'})
    if fb_app_tag:
        res_dict['news_fb_app_id'] = str(fb_app_tag['content'])

    fb_page_tag = soup.find('meta', attrs = {'property':'fb:admins'})
    if fb_page_tag:
        res_dict['news_fb_page'] = str(fb_page_tag['content'])

    #Optional
    keywords_tag = soup.find('meta', attrs={'name': 'news_keywords'})
    if keywords_tag:
        res_dict['news_keywords'] = keywords_tag['content']

    description_tag = soup.find('meta', attrs = {'name': 'description'})
    if description_tag:
        res_dict['news_description'] = description_tag['content']

    time_tag = soup.find('meta', attrs = {'property': 'article:published_time'})

    if time_tag:
        try:
            d1 = datetime.datetime.strptime(time_tag.get('content'), "%Y-%m-%dT%H:%M:%S+08:00") 
            d1 -= datetime.timedelta(hours=8)
            db_date_format = '%Y-%m-%d %H:%M:%S'
            date_res = d1.strftime(db_date_format)
            res_dict['news_published_date'] = date_res
        except Exception as e1:
            try:
                d1 = datetime.datetime.strptime(time_tag.get('content'), "%Y-%m-%dT%H:%M:%SZ") 
                d1 -= datetime.timedelta(hours=8)
                db_date_format = '%Y-%m-%d %H:%M:%S'
                date_res = d1.strftime(db_date_format)
                res_dict['news_published_date'] = date_res
            except Exception as e2:
                print(e2)
                content_parser.logger.info('NewsTalk date error {}, URL: {}'.format(e2, url))

    article_body_tag = soup.find('div', attrs = {'itemprop':'articleBody'})
    #print(article_body_tag)
    temp_content, links, links_descs = [], [], []
    if article_body_tag:
        p_tags = article_body_tag.find_all('p', attrs = {'class': None})
        a_tags = article_body_tag.find_all('a')
        if p_tags:
            for p in p_tags:
                if p.text:
                    temp_content.append(p.text.strip())

        if len(a_tags):
            for a in a_tags:
                if len(a):
                    if a['href'] == '#':
                        continue
                    if a.get_text().strip() and 'www' in a['href']:
                        links.append(a['href'])
                        links_descs.append(a.get_text().strip())
            res_dict['news_related_url'] = links
            res_dict['news_related_url_desc'] = links_descs
    content = '\n'.join(temp_content).strip()
    if content:
        res_dict['news'] = content

    if not res_dict or 'news' not in res_dict:
        return
        content_parser.logger.error('NewsTalk url: {} did not process properly'.format(url))

    return res_dict

In [39]:
print_util(newstalk_test_urls, newstalk_content_processor)


News_title: 巴西成武漢肺炎重災區 急招2千人試牛津大學疫苗
News_category: 國際
News_fb_app_id: 241741912846250
News_fb_page: 100007902165474
News_keywords: 牛津大學,巴西,武漢肺炎,疫苗人體測試
News_description: 中國武漢肺炎肆虐全球，目前巴西疫情嚴峻，確診人數突破55萬人、死亡逾3萬1000人，成為僅次於美國的第2重災區。各國都在加緊疫苗研發，外電報導，英國牛津大學研製的實驗疫苗將於6月中旬在巴西進行試驗，這是英國以外第一個參加該研究的國家，首批將招募2000名志願者參與。巴西聖保羅聯邦大學（Federal
News_published_date: 2020-06-04 02:33:15
News: 中國武漢肺炎肆虐全球，目前巴西疫情嚴峻，確診人數突破55萬人、死亡逾3萬1000人，成為僅次於美國的第2重災區。各國都在加緊疫苗研發，外電報導，英國牛津大學研製的實驗疫苗將於6月中旬在巴西進行試驗，這是英國以外第一個參加該研究的國家，首批將招募2000名志願者參與。
巴西聖保羅聯邦大學（Federal University of Sao Paulo）證實該消息，校方表示，該疫苗的2000名志願者將從本週開始招募。校長斯邁利（Soraya Smaili）告訴法新社，志願者鎖定在18至55歲間的第一線防疫醫療人員，處於高風險，但沒有感染過新型冠狀病毒（COVID-19）。她補充說在巴西測試疫苗很重要，因為境內疫情正加速擴大中。
該大學透露，測試將於6月的第2個星期在巴西開始，然後擴展到其他國家。
這款疫苗由牛津大學與英國製藥集團阿斯利康（AstraZeneca）合作開發，美國生物技術公司「Moderna 」也有參與。報導指4月已經開始人體試驗，但牛津大學詹納研究所（Jenner Institute）所長暨武漢肺炎疫苗研發團隊共同領導人希爾（Adrian Hill）擔心，因為病毒在英國社區傳播率較低，試驗成功的機率可能只有50%，而巴西已經成為目前疫情中心，因此適合進行測試。
巴西衛生部與國家衛生監督局（Anvisa）已經同意該項疫苗測試，將在聖保羅與里約熱內盧分別招募1000名志願者。據世界衛生組織（WHO）公布的資料，目前全球至少有133款疫苗正